# Prerequisites

## Install the required python libaries

In [1]:
!pip install transformers datasets accelerate peft bitsandbytes trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.0 MB/s eta 0:00:00


## Login to the 🤗 Hub

In [23]:
from huggingface_hub import login
login()

# Dataset preparations

## Load dataset


To fine tune the model, we need to load a dataset.
Here you would use your own dataset instead of **HuggingFaceH4/helpful-instructions**.
We will only use the first 5% of the dataset to reduce the training time.

In [3]:
from datasets import load_dataset, DatasetDict
instruct_tune_dataset = load_dataset("HuggingFaceH4/helpful-instructions", split='train[:5%]')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/20.8M [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/12.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/147706 [00:00<?, ? examples/s]

## Preprocessing dataset

Each model has a specific input format. We need to preprocess the dataset to match the model's input format. You can find the model's input format in the model's documentation. The easiest way is to follow the reference code provided by the model's author. Have a look at the [Mistral-7B-Instruct reference implementation.](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/blob/main/README.md#instruction-format)

In [4]:
# Function to format dataset for Mistral
def format_for_mistral(examples):
    # Reformat into the required text format
    formatted_texts = [
        f"<s>[INST] {instruction} [/INST] {demonstration}</s>"
        for instruction, demonstration in zip(examples["instruction"], examples["demonstration"])
    ]
    return {"text": formatted_texts}

# Apply the formatting function
instruct_tune_dataset_mistral = instruct_tune_dataset.map(format_for_mistral, batched=True)


Map:   0%|          | 0/7385 [00:00<?, ? examples/s]

We need to make a train, test, eval split of the dataset

In [5]:
# 70% train, 10% eval, 20% test
train_test_split = instruct_tune_dataset_mistral.train_test_split(test_size=0.30)
eval_test_split = train_test_split['test'].train_test_split(test_size=0.66)

# Create the final DatasetDict with the splits
final_dataset = DatasetDict({
    'train': train_test_split['train'], # 70% of the original dataset
    'eval': eval_test_split['train'],   # 10% of the original dataset
    'test': eval_test_split['test']     # 20% of the original dataset
})

Have a look at the final dataset

In [6]:
final_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'demonstration', 'meta', 'text'],
        num_rows: 5169
    })
    eval: Dataset({
        features: ['instruction', 'demonstration', 'meta', 'text'],
        num_rows: 753
    })
    test: Dataset({
        features: ['instruction', 'demonstration', 'meta', 'text'],
        num_rows: 1463
    })
})

# Mistral 7B fine-tuning


## Import necessary libraries


In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import TrainingArguments
from trl import SFTTrainer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

## Bits and Bytes settings for 4-bit quantization

In [8]:
# Define the model name to be used
model_name = "mistralai/Mistral-7B-v0.1"

4-bit quantization is a technique used to reduce the memory footprint and computational cost to fit the model on the T4 GPU.

In [9]:
# Configure Bits and Bytes settings for 4-bit quantization
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,                      # Load model in 4-bit precision
    bnb_4bit_quant_type="nf4",              # Use NF4 quantization type
    bnb_4bit_use_double_quant=True,         # Enable double quantization
    bnb_4bit_compute_dtype=torch.bfloat16   # Use bfloat16 for computation
)

In [12]:
# Load the pre-trained model with the specified quantization settings
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',                    # Automatically map model to available devices (e.g., GPU)
    quantization_config=nf4_config,       # Apply the quantization configuration
    use_cache=False                       # Disable caching for training
)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [13]:
# Load the tokenizer associated with the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token     # Set padding token to end-of-sequence token
tokenizer.padding_side = "right"              # Pad on the right side

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [14]:
# Prepare the model for k-bit training
model = prepare_model_for_kbit_training(model)

## PEFT (Parameter Efficient Fine-Tuning)

 LoRA (Low-Rank Adaptation), allows for effective fine-tuning of large models with fewer parameters. LoRA enables fine-tuning by only modifying a small subset of parameters while keeping the majority of the model fixed.

 You can find out more about PEFT and different techniques other the LoRA to reduce the memory footprint [here](https://huggingface.co/docs/peft/en/index)

In [15]:
# Define PEFT (Parameter Efficient Fine-Tuning) configuration
peft_config = LoraConfig(
    lora_alpha=16,                  # Scaling factor for LoRA
    lora_dropout=0.1,               # Dropout rate for LoRA
    r=64,                           # Rank for LoRA
    bias="none",                    # No bias terms used
    task_type="CAUSAL_LM"           # Specify the task type (Causal Language Model)
)

In [16]:
# Prepare the model for PEFT
model = get_peft_model(model, peft_config)

You can check the if the PEFT settings are applied correctly by looking at the amount of trainable parameters.

In [17]:
def print_train_params(model):
    # Calculate total and trainable parameters
    all_param = sum(param.numel() for param in model.parameters())
    trainable_params = sum(param.numel() for param in model.parameters() if param.requires_grad)

    # Print summary of parameters
    print(
        f"trainable params: {trainable_params} \n all params: {all_param} \n trainable%: {100 * trainable_params / all_param:.2f}"
    )

# Print the training parameters of the model
print_train_params(model)

## Training Parameters

In [18]:
# Define training arguments for the model
args = TrainingArguments(
    output_dir="mistral_instruct_generation",     # Directory to save model outputs
    warmup_steps=1,                               # Number of warmup steps for learning rate
    per_device_train_batch_size=4,                # Batch size per device during training
    gradient_accumulation_steps=1,                # Steps for gradient accumulation
    gradient_checkpointing=True,                  # Enable gradient checkpointing to save memory
    max_steps=30,                                # Total training steps
    learning_rate=1e-4,                           # Learning rate for the optimizer
    fp16=True,                                    # Use mixed precision training
    fp16_full_eval=True,                          # Use mixed precision during evaluation
    optim="paged_adamw_8bit",                     # Use 8-bit AdamW optimizer
    logging_steps=10,                            # Steps between logging
    logging_dir="./logs",                         # Directory for storing logs
    save_strategy="steps",                        # Strategy for saving model checkpoints
    save_steps=10,                               # Steps between model saves
    eval_strategy="steps",                        # Strategy for evaluation
    eval_steps=10,                               # Steps between evaluations
    do_eval=True,                                 # Perform evaluation during training
    report_to="none",                             # Do not report to any tracking tool (i.e. wandb etc.)
)

In [19]:
# Set maximum sequence length for training
max_seq_length = 512

In [20]:
# Initialize the trainer with the model, tokenizer, and training arguments
trainer = SFTTrainer(
    model=model,                                # The model to train
    peft_config=peft_config,                    # Configuration for PEFT
    max_seq_length=max_seq_length,              # Maximum sequence length for input
    tokenizer=tokenizer,                        # Tokenizer for processing text
    packing=True,                               # Enable packing for efficiency
    args=args,                                  # Training arguments
    dataset_text_field="text",                  # Field in dataset that contains text field that we created in the preprocessing step
    train_dataset=final_dataset["train"],       # Training dataset
    eval_dataset=final_dataset["eval"]          # Evaluation dataset
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, packing, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:195: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will overri

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:421: UserWarning: You passed `packing=True` to the SFTTrainer/SFTConfig, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


A few notes on the chosen settings:

- **Learning Rate**: Start with a low learning rate, like `1e-4` or `1e-5`. Fine-tuning a pre-trained LLM requires subtle adjustments rather than aggressive changes.
- **Batch Size**: Due to GPU memory limitations, set a small batch size, e.g., `4`, with gradient accumulation to simulate a larger batch size.
- **Gradient Checkpointing**: Enables storing less memory by trading it off with more compute, which is crucial for fitting large models into memory.
- **Mixed Precision (FP16)**: It helps save memory and can speed up training.


## Start the Fine-Tuning job


In [21]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
10,2.462600,2.295489
20,2.144200,1.984046
30,1.966200,1.914127


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=30, training_loss=2.1909634272257485, metrics={'train_runtime': 433.6233, 'train_samples_per_second': 0.277, 'train_steps_per_second': 0.069, 'total_flos': 2631323961262080.0, 'train_loss': 2.1909634272257485, 'epoch': 0.13574660633484162})

## Save the fine tuned model

You can save the model in different ways. Here we are going to push the model to 🤗 Hub.

Have a look at our [enterpise version](https://huggingface.co/enterprise)

In [24]:
trainer.save_model("mistral7b-hf-cloud-mle")
trainer.push_to_hub("Mystorius/mistral7b-hf-cloud-mle")

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/109M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Mystorius/mistral_instruct_generation/commit/186fa65d1b2e3fe6cc778baa02136b5bd72dcf76', commit_message='Mystorius/mistral7b-hf-cloud-mle', commit_description='', oid='186fa65d1b2e3fe6cc778baa02136b5bd72dcf76', pr_url=None, pr_revision=None, pr_num=None)

# Evaluation

After fine-tuning, you have several options to evaluate your model's performance. In general, evaluation for LLM's is hard, and there is no "go-to" method, but you have a few choices:

1.   Quantitative Metrics (Train & Eval Loss)
2.   Qualitative Evaluation or Human Evaluation. Have domain experts or employees familiar with the company’s use cases evaluate the responses based on correctness, fluency, and relevance.


To learn more about LLM Evaluation, have a look at our [Blog](https://huggingface.co/blog/clefourrier/llm-evaluation)

Below is an exaple for quantitive metrics using our [🤗 Evaluate](https://huggingface.co/docs/evaluate/en/index) libary.

In [28]:
trainer.evaluate()

{'eval_loss': 1.9140832424163818,
 'eval_runtime': 69.1151,
 'eval_samples_per_second': 1.895,
 'eval_steps_per_second': 0.246,
 'epoch': 0.13574660633484162}

To use the model and run it agains our test dataset, we need to tokenize our *instruction* column so it can be used for the models forward() function.

In [97]:
# Define a function to preprocess the dataset (tokenizing the 'instruction')
def preprocess_function(examples):
    # Tokenize 'instruction' as input
    tokenized_inputs = tokenizer(examples['instruction'], truncation=True, padding=True)
    return tokenized_inputs

# Apply the preprocessing to the test dataset
tokenized_test_dataset = final_dataset["test"].map(preprocess_function, batched=True)

# Now we remove the unnecessary columns such as 'instruction', 'meta', 'text', but we keep 'labels'
tokenized_test_dataset = tokenized_test_dataset.remove_columns(["instruction", "meta", "text"])

# Let use 5 rows as an example
tokenized_test_dataset = tokenized_test_dataset.select(range(5))

In [102]:
predictions = trainer.predict(
    test_dataset=tokenized_test_dataset
)

In [103]:
# Extract the first 5 original inputs (instructions) and ground truth outputs (demonstrations)
original_instructions = final_dataset["test"]["instruction"][:5]  # Input instructions
ground_truth_outputs = final_dataset["test"]["demonstration"][:5]  # Expected outputs

In [109]:
# Decode the predicted tokens back into text
predicted_tokens = predictions.predictions.argmax(axis=-1)
decoded_predictions = tokenizer.batch_decode(predicted_tokens, skip_special_tokens=True)

In [105]:
# Print input, ground truth, and prediction for each of the 5 examples
for i in range(5):
    print(f"Input (Instruction): {original_instructions[i]}")
    print(f"Ground Truth (Demonstration): {ground_truth_outputs[i]}")
    print(f"Generated Output: {decoded_predictions[i]}")
    print("-" * 50)  # Separator

#Going Bigger


Let’s assume the results from the Mistral-7B are not satisfactory and you want to try out a larger open-source model like LLaMA-70B. Here are some considerations:

1. **Memory Scaling**: LLaMA-70B has 10x more parameters than Mistral-7B, requiring roughly 10x of memory for training. This would exceed a single T4 GPU's capability by far. To fine-tune such large models, you would need:
   - **Multi-GPU setup**: You could explore distributed training across multiple GPUs or even TPUs.
   - **Memory-efficient techniques**: Use **deepspeed**, **zero-optimization**, and **tensor parallelism** to handle the larger model size.

2. **Longer Training Time**: The training duration will scale accordingly with the model size and dataset complexity.

3. **Costs**: Larger models require expensive cloud infrastructure or optimized hardware to support fine-tuning.

While fine-tuning a large model like LLaMA-70B would provide additional capabilities, it might to expensive and time consuming.


## Fine-Tuning as a Service

If managing the fine-tuning process yourself seems daunting, you can opt for fine-tuning as a service offered by cloud providers like Amazon Web Services (AWS) or Google Cloud Platform (GCP). These platforms provide managed solutions that simplify the process of training large models:

- **AWS Bedrock**: You can find our more [here](https://aws.amazon.com/blogs/aws/customize-models-in-amazon-bedrock-with-your-own-data-using-fine-tuning-and-continued-pre-training/)
  
- **GCP Vertext AI**: You can find out more [here](https://cloud.google.com/vertex-ai/generative-ai/docs/models/tune-models)

Utilizing these services can save you time and effort, allowing you to focus on data preparation and model evaluation rather than the complexities of infrastructure management.